In [1]:
import numpy as np

import tensorflow as tf

!pip install -q tensorflow-hub
!pip install -q tensorflow-datasets
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

c:\Users\gka06\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Version:  2.12.0
Eager mode:  True
Hub version:  0.14.0
GPU is NOT AVAILABLE


In [2]:
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

In [3]:
#embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
embedding = hub.load("https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2")
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)

In [4]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [5]:
hub_layer(train_examples_batch)[0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 1.1501508 ,  0.078013  ,  0.09266154,  0.28336146,  0.00967164,
       -0.14918603,  0.3356657 , -0.3502448 , -0.00828831, -0.0187714 ,
       -0.03330697, -0.6330942 , -0.37542117, -0.27773282, -0.09661756,
        0.17255366, -0.13367656,  0.03807658, -0.27513817,  0.4947623 ,
        0.03930517,  0.13449624, -0.2707282 ,  0.01789428, -0.2410716 ,
        0.02710899,  0.10233343, -0.10662853,  0.05242986,  0.11917052,
       -0.00667078,  0.3392315 ,  0.11301497,  0.10684261,  0.39157125,
       -0.18953632, -0.17400031, -0.10644417, -0.13420047,  0.17358392,
       -0.27769503, -0.04335914, -0.03915006, -0.19834091,  0.02748548,
        0.276703  ,  0.14070286, -0.3142561 ,  0.02475694, -0.16730827,
        0.08217403, -0.09568419, -0.25733578,  0.3729147 , -0.22105993,
       -0.21443687, -0.05832561,  0.15003687, -0.16772385, -0.080281  ,
       -0.21137346, -0.0039831 ,  0.06347676,  0.06651689,  0.09252462,
        0.113278

In [6]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 128)               124642688 
                                                                 
 dense (Dense)               (None, 32)                4128      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 124,646,849
Trainable params: 124,646,849
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

In [ ]:
results = model.evaluate(test_data.batch(32), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))